In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_dir = '.'
conf_f = os.path.join(data_dir, 'f_channel_1.csv')  # old: f_channel.csv
conf_c = os.path.join(data_dir, 'c_channel_1.csv')  # old: c_channel.csv

In [3]:
conf_f_df = pd.read_csv(conf_f)
conf_f_df = conf_f_df.iloc[:,1:]
conf_f_df

,source_dataset,source_channel,transfer_dataset,transfer_channel,epoch
0,mass_ss1,EEG F3,mass_ss1,EEG F4,30
1,mass_ss1,EEG F4,mass_ss3,EEG F4,30
2,mass_ss1,EEG F4,isruc1,EEG F4 A1,30
3,mass_ss1,EEG F4,SleepEDF,EEG Fpz Cz,30
4,mass_ss1,EEG C4,mass_ss1,EEG F4,30
...,...,...,...,...,...
74,shhs1_normal,EEG C4 A1,SleepEDF,EEG Fpz Cz,30
75,shhs1_osa,EEG C4 A1,mass_ss1,EEG F4,30
76,shhs1_osa,EEG C4 A1,mass_ss3,EEG F4,30
77,shhs1_osa,EEG C4 A1,isruc1,EEG F4 A1,30


In [4]:
conf_c_df = pd.read_csv(conf_c)
conf_c_df = conf_c_df.iloc[:,1:]
conf_c_df

,source_dataset,source_channel,transfer_dataset,transfer_channel,epoch
0,mass_ss1,EEG F4,mass_ss1,EEG C4,30
1,mass_ss1,EEG F4,mass_ss3,EEG C4,30
2,mass_ss1,EEG F4,isruc1,EEG C4 A1,30
3,mass_ss1,EEG F4,shhs1_normal,EEG C4 A1,30
4,mass_ss1,EEG F4,shhs1_osa,EEG C4 A1,30
...,...,...,...,...,...
100,shhs1_osa,EEG C4 A1,mass_ss1,EEG C4,30
101,shhs1_osa,EEG C4 A1,mass_ss3,EEG C4,30
102,shhs1_osa,EEG C4 A1,isruc1,EEG C4 A1,30
103,shhs1_osa,EEG C4 A1,shhs1_normal,EEG C4 A1,30


In [5]:
local_data_dir = './processed'
map_db_dir = {
    'mass_ss1': 'mass_ss1',
    'mass_ss2': 'mass_ss2',
    'mass_ss3': 'mass_ss3',
    'mass_ss4': 'mass_ss4',
    'mass_ss5': 'mass_ss5',
    'SleepEDF': 'sedf_sc',
    'isruc1': 'isruc_sg1', 
    'shhs1_normal': 'shhs1_nor',
    'shhs1_osa': 'shhs1_osa',
}

In [6]:
def ch2arguments(ch):
    specify_chs = ''
    if '-' not in ch and len(ch) == 2:
        specify_chs = f'--channels {ch}-CLE --alt_channels {ch}-LER'
    else:
        specify_chs = f'--channels {ch}'
    return specify_chs

In [7]:
base_cmd = 'cmd'
commands = []
count = 0
for conf_df in [conf_f_df, conf_c_df]:
    base_df = conf_df.groupby(by=['source_dataset','source_channel']).size().reset_index().rename(columns={0:'count'})
    for _, b_row in base_df.iterrows():
        # Base dataset
        b_db = str(b_row['source_dataset']).lower()
        b_ch = str(b_row['source_channel']).replace('EEG ','').replace(' ','-')
        b_dir = os.path.join(local_data_dir, map_db_dir[b_row['source_dataset']], 'views/fixed_split')

        # Base dataset
        each_base_df = conf_df[(conf_df['source_dataset'] == b_row['source_dataset']) & (conf_df['source_channel'] == b_row['source_channel'])]
        tran_df = each_base_df.groupby(by=['transfer_dataset']).size().reset_index().rename(columns={0:'count'})
        
        # Initialize command
        src_proj = f"{b_db.lower().replace(' ','_')}_{b_ch.lower()}_project"
        init_cmd = f"ut init --name {src_proj} --model utime --data_dir {b_dir}"
        specify_chs = ch2arguments(b_ch)
        train_cmd = f"ut train --num_gpus=1 --force_gpu=0 {specify_chs}"
        print(init_cmd)
        print(f"cd {src_proj}")
        print(train_cmd)
        print(f"cd ..")
        
        for _, t_row in tran_df.iterrows():
            t_chs = each_base_df[each_base_df['transfer_dataset'] == t_row['transfer_dataset']]['transfer_channel'].values
            count += len(t_chs)
            t_chs = '|'.join(t_chs)
            t_chs = t_chs.replace('EEG ','').replace(' ', '-')
            t_db = str(t_row['transfer_dataset']).lower()
            t_dir = os.path.join(local_data_dir, map_db_dir[t_row['transfer_dataset']], 'views/fixed_split')
            
            tgt_proj = f"tf_{src_proj}__to__{t_db.lower().replace(' ','_')}_{t_chs.lower()}_project"
            tf_init_cmd = f"ut init --name {tgt_proj} --model utime --data_dir {t_dir}"
            specify_chs = ch2arguments(t_chs)
            weight_from = f"{src_proj}/model/model_weights.h5"
            transfer_cmd = f"ut train --num_gpus=1 --force_gpu=0 {specify_chs} --initialize_from ../{weight_from}"
            print(tf_init_cmd)
            print(f"cd {tgt_proj}")
            print(transfer_cmd)
            print(f"cd ..")

ut init --name sleepedf_fpz-cz_project --model utime --data_dir ./processed/sedf_sc/views/fixed_split
cd sleepedf_fpz-cz_project
ut train --num_gpus=1 --force_gpu=0 --channels Fpz-Cz
cd ..
ut init --name tf_sleepedf_fpz-cz_project__to__isruc1_f4-a1_project --model utime --data_dir ./processed/isruc_sg1/views/fixed_split
cd tf_sleepedf_fpz-cz_project__to__isruc1_f4-a1_project
ut train --num_gpus=1 --force_gpu=0 --channels F4-A1 --initialize_from ../sleepedf_fpz-cz_project/model/model_weights.h5
cd ..
ut init --name tf_sleepedf_fpz-cz_project__to__mass_ss1_f4_project --model utime --data_dir ./processed/mass_ss1/views/fixed_split
cd tf_sleepedf_fpz-cz_project__to__mass_ss1_f4_project
ut train --num_gpus=1 --force_gpu=0 --channels F4-CLE --alt_channels F4-LER --initialize_from ../sleepedf_fpz-cz_project/model/model_weights.h5
cd ..
ut init --name tf_sleepedf_fpz-cz_project__to__mass_ss3_f4_project --model utime --data_dir ./processed/mass_ss3/views/fixed_split
cd tf_sleepedf_fpz-cz_projec